In [215]:
import sympy as sp
import numpy as np
import re


def func_from_expression(exp):
    try:
        expression = re.sub(r"(\d+)([a-zA-Z])", r"\1*\2", exp)
        f = sp.sympify(expression)
        return f
    except:
        raise ValueError("Неверный ввод. Введите корректное математическое выражение.")


def read_math_func(funcs: list[str]):

    print("Введите номер уравнения или введите свое: ")
    for index, fun in enumerate(funcs):
        print(f"{index+1}. {fun}")
    ans = input("Ввод: ")

    try:
        if ans.isdigit():
            expression = funcs[int(ans) - 1]
        else:
            expression = re.sub(r"(\d+)([a-zA-Z])", r"\1*\2", ans)

        f = sp.sympify(expression)
        print("Исходная функция:", f)
        return f
    except:
        raise ValueError("Неверный ввод. Введите корректное математическое выражение.")

In [230]:
f = read_math_func(["x^2 + 3*x", "x**11", "x^3 - 1.89*x^2 - 2*x + 1.76"])

Введите номер уравнения или введите свое: 
1. x^2 + 3*x
2. x**11
3. x^3 - 1.89*x^2 - 2*x + 1.76
Исходная функция: x**3 - 1.89*x**2 - 2*x + 1.76


In [ ]:
def method(methods: list[str]):

    print("Введите номер метода: ")
    for index, method in enumerate(methods):
        print(f"{index+1}. {method}")

    ans = input("Ввод: ")

    if not ans.isdigit() or int(ans) - 1 not in [
        index for index, _ in (enumerate(methods))
    ]:
        raise ValueError("Неверный ввод. Метода с указаным номером не существует")
    return methods[int(ans) - 1]

In [ ]:
methods = ["newton", "half-division", "iteration", "system-iteration"]

print(method(methods))

Введите номер метода: 
1. newton
2. half-division
3. iteration
4. system-iteration
iteration


In [272]:
def half_div(f, a: int, b: int, eps: int):
    x_symbol = sp.Symbol("x")

    while True:
        x = (a + b) / 2

        val_x = f.subs(x_symbol, x)
        val_a = f.subs(x_symbol, a)

        if val_a * val_x > 0:
            a = x
        else:
            b = x

        print(a, b, x, val_a, f.subs(x_symbol, b), val_x, abs(val_x), sep="; ")
        if abs(val_x) < eps:
            break
    return x

In [273]:
half_div(f, -2, -1, 0.1)

-1.5; -1; -1.5; -9.80000000000000; 0.870000000000000; -2.86750000000000; 2.86750000000000
-1.25; -1; -1.25; -2.86750000000000; 0.870000000000000; -0.646250000000000; 0.646250000000000
-1.25; -1.125; -1.125; -0.646250000000000; 0.194140625000000; 0.194140625000000; 0.194140625000000
-1.1875; -1.125; -1.1875; -0.646250000000000; 0.194140625000000; -0.204755859375000; 0.204755859375000
-1.15625; -1.125; -1.15625; -0.204755859375000; 0.194140625000000; -7.44628906250711e-5; 7.44628906250711e-5


-1.15625

In [ ]:
def newton(f, a: int, b: int, eps: int):
    x_symbol = sp.Symbol("x")

    df = sp.diff(f, x_symbol)
    ddf = sp.diff(df, x_symbol)

    if ddf.subs(x_symbol, a) * f.subs(x_symbol, a) > 0:
        x = a
    else:
        x = b

    while True:
        val_x = f.subs(x_symbol, x)
        dval_x = df.subs(x_symbol, x)
        x = x - val_x / dval_x

        if abs(val_x) < eps:
            break
    return x.evalf()

In [ ]:
newton(f, -4, -1, 0.0001)

-1.15623833108674

In [253]:
def iter(f, a, b, eps):
    x_symbol = sp.Symbol("x")
    df = sp.diff(f, x_symbol)
    df_a, df_b = df.subs(x_symbol, a), df.subs(x_symbol, b)

    lam = -1 / max(df_a, df_b) * (df_a / abs(df_a))

    x = a
    while True:
        new = x + lam * f.subs(x_symbol, x)

        print(
            x,
            new,
            x + lam * f.subs(x_symbol, x),
            f.subs(x_symbol, x),
            abs(new - x),
            sep="; ",
        )
        if abs(new - x) < eps:
            return new
        x = new

In [254]:
iter(f, 2, 3, 0.1)

2; 2.13177159590044; 2.13177159590044; -1.80000000000000; 0.131771595900439
2.13177159590044; 2.23461377962424; 2.23461377962424; -1.40482422966716; 0.102842183723804
2.23461377962424; 2.30697095996831; 2.30697095996831; -0.988399083499910; 0.0723571803440639


2.30697095996831

In [348]:
def system_iter(funcs, init_approx, a, b, eps):

    x_symbol = sp.Symbol("x")
    y_symbol = sp.Symbol("y")

    while True:
        tmp = []
        isEnd = True
        for func in funcs:
            v = func.subs({x_symbol: init_approx[0], y_symbol: init_approx[1]})
            tmp.append(v)
        for i in range(len(tmp)):
            if not (abs(tmp[i] - init_approx[i]) <= eps):
                isEnd = False
                break
        init_approx = tmp
        if isEnd:
            break

    return init_approx

In [351]:
f1, f2 = func_from_expression("3 + cos(y)"), func_from_expression("0.5 - cos(x-1)")

[i.evalf() for i in system_iter([f1, f2], [3, 1], -2, -1, 0.001)]

[3.35545609519338, 1.20666240376498]

In [315]:
def secant(f, a: int, b: int, eps: int):
    x_symbol = sp.Symbol("x")

    df = sp.diff(f, x_symbol)
    ddf = sp.diff(df, x_symbol)

    if ddf.subs(x_symbol, a) * f.subs(x_symbol, a) > 0:
        x = a
        prev = x - 0.1
    else:
        x = b
        prev = x + 0.1

    while True:
        val_x = f.subs(x_symbol, x)
        dval_x = f.subs(x_symbol, prev)
        tmp_prev = prev
        x, prev = x - (x - prev) / (val_x - dval_x) * val_x, x

        print(
            tmp_prev,
            prev,
            x,
            f.subs(x_symbol, x),
            abs(x - prev),
            sep="; ",
        )
        if abs(val_x) < eps:
            break
    return x.evalf()

In [317]:
secant(f, 0, 1, 0.01)

1.1; 1; 0.575028206092516; 0.175138383659651; 0.424971793907484
1; 0.575028206092516; 0.632055778047911; -0.00665336232929226; 0.0570275719553954
0.575028206092516; 0.632055778047911; 0.629968636539285; 6.07139407104995e-6; 0.00208714150862555


0.629968636539285

In [339]:
import math

def system_newton(x, y, eps):
    print((((0.8 *y + 4.8 * x * y - 4 * y * math.sin(x+y))/(math.cos(x + y))) - 1 + x **2 + 2 * y * y) / (4 * y - 2 *x))
    

    while True:
        d_x = (((0.8 *y + 4.8 * x * y - 4 * y * math.sin(x+y))/(math.cos(x + y))) - 1 + x **2 + 2 * y * y) / (4 * y - 2 *x)
        d_y = (1 - x**2 - 2 * y**2 - 2 * x * d_x) / (4 * y)
        x1 = x + d_x
        y1 = y + d_y
        if abs(x1 - x) <= eps or abs(y1 - y) <= eps:
            break
        x = x1
        y = y1

    return d_x, d_y
# cos(-0.03322430890050499)x + cos(-0.03322430890050499)y = 0.2 + 1.2 * 0.0002180488656566765 - sin(-0.03322430890050499)
# 2 * 0.0002180488656566765 *x + 4 * (-0.03344235776616167)* y = 1 - 0.0002180488656566765 * 0.0002180488656566765 - 2 * 0.03344235776616167 * 0.03344235776616167

In [340]:
system_newton(1, 1, 0.01)

-1.358314566299628


KeyboardInterrupt: 